# List Experiments in Current Namespace

In [29]:
!kubectl get experiments

NAME                     STATUS      AGE
mxnet-mnist-pvc-1        Succeeded   19m
mxnet-mnist-pvc-latest   Succeeded   8h


#  Delete Experiment If exists

In [25]:
!kubectl delete experiments mxnet-mnist-pvc

experiment.kubeflow.org "mxnet-mnist-pvc" deleted


# Create New Experiment

In [ ]:
# %%bash
cat <<EOF | kubectl apply -f -
apiVersion: "kubeflow.org/v1alpha3"
kind: Experiment
metadata:
  namespace: dev1
  name: mxnet-mnist-pvc
spec:
  objective:
    type: maximize
    goal: 0.99
    objectiveMetricName: Validation-accuracy
    additionalMetricNames:
      - Train-accuracy
  algorithm:
    algorithmName: random
  parallelTrialCount: 3
  maxTrialCount: 13
  maxFailedTrialCount: 3
  parameters:
    - name: --lr
      parameterType: double
      feasibleSpace:
        min: "0.01"
        max: "0.03"
    - name: --num-layers
      parameterType: int
      feasibleSpace:
        min: "2"
        max: "5"
    - name: --optimizer
      parameterType: categorical
      feasibleSpace:
        list:
          - sgd
          - adam
          - ftrl
  trialTemplate:
    goTemplate:
        rawTemplate: |-
          apiVersion: batch/v1
          kind: Job
          metadata:
            name: {{.Trial}}
            namespace: {{.NameSpace}}
          spec:
            template:
              spec:
                volumes:
                  - name: dataset
                    persistentVolumeClaim:
                      claimName: mxnet-mnist-pvc
                containers:
                  - name: training-container
                    image: ilovepython/katib-mxnet-mnist-pvc:v0.1
                    command:
                      - 'python3'
                      - '/opt/mxnet-mnist/mxnet-mnist.py'
                      - '--batch-size=64'
                      - '--data-dir=/dataset/MXNET-MNIST'
                      {{- with .HyperParameters}}
                      {{- range .}}
                      - "{{.Name}}={{.Value}}"
                      {{- end}}
                      {{- end}}
                    volumeMounts:
                      - mountPath: "/dataset"
                        name: dataset
                restartPolicy: Never
EOF

# List All Experiments

In [31]:
!kubectl get experiments mxnet-mnist-pvc

NAME                STATUS      AGE
mxnet-mnist-pvc-1   Succeeded   21m


# List All Pods

In [24]:
!kubectl get pods

NAME                                             READY   STATUS    RESTARTS   AGE
dataaccess                                       2/2     Running   0          8h
mxnet-mnist-pvc-1-5hkrnjsv-d8sd4                 2/2     Running   0          63s
mxnet-mnist-pvc-1-6rkl5wss-9qnnb                 2/2     Running   0          63s
mxnet-mnist-pvc-1-r4lh5hrm-lglgq                 2/2     Running   0          63s
mxnet-mnist-pvc-1-random-8455fcb9ff-bzp52        1/1     Running   0          104s
mxnet-mnist-pvc-latest-random-6d9d979874-9tj7m   1/1     Running   0          8h
mxnet-mnist-pvc-random-74dc46d967-xhpt4          1/1     Running   0          25m
testing-0                                        2/2     Running   0          40m
